## Build history of ETF shares outstanding and of commodity COTs

### For ETF histories:
Several companies that manage ETFs provide histores of the NAVs and shares outstanding of their ETF products.  This notebook fetchs that history data from those websites, and assembles a Pandas DataFrame with columns:
1. symbol: like SPY or XLE
2. date: a datetime.datetime object
3. nav: the funds nav for that date
4. shares: the shares outstanding at the end of that date
5. pc: the percent change of those shares outstanding
6. share_diff: the absolute change of those shares outstanding

### For COT histories:
1. Retrieve data from the CFTC website (www.cft.gov/files);
2. Extract data from the most important Commercial and Non Commercial long and short columns;
3. Create "net" columns for each important category;
4. Merge this data with the ETF history data created in the previous steps


In [ ]:
import pandas as pd
import numpy as np
import os, sys
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import zipfile
import urllib.request
from PIL import Image
def str_to_date(d):
    try:
        dt = datetime.datetime.strptime(str(d),'%Y-%m-%d')
    except:
        return None
    return dt

# Make important folders
TEMP_FOLDER = './temp_folder'
try:
    os.mkdir(TEMP_FOLDER)
except:
    pass
try:
    os.mkdir(f'{TEMP_FOLDER}/cot')
except:
    pass
try:
    os.mkdir(f'{TEMP_FOLDER}/zip')
except:
    pass


___
## First, decide if you want to re-create the ETF and COT data, or just retrieve the previously saved data DataFrames.

In [ ]:
CREATE_ETF_DATA = False
CREATE_COT_DATA = False
etf_save_path = './etf_cap_hist.csv'
cot_save_path = './cot_history.csv'

___
### Retrieve data from iShares
Currently, you must physically download each etf history because iShares does not support csv downloads for the commoditiy ETFs.  For other funds, they do.

As an example, to process the iShares SLV (silver etf) download:
1. Download the file from the url "https://www.ishares.com/us/products/239855/ishares-silver-trust-fund/1521942788811.ajax?fileType=xls&fileName=iShares-Silver-Trust_fund&dataType=fund" to a local folder;
2. That url is actually an xml file which Microsoft Excel can convert into an xls workbook;
3. Open that file in Microsoft Excel;
4. Save the "Historical" worksheet as a csv with the file name 'etf_history.csv', where etf is something like "slv".
  * **Make sure you save the file in the folder that contains this jupyter Noteboook**
5. Execute the code below.
___

In [ ]:
if CREATE_ETF_DATA:
    ishares_symbol_list = ['SLV']
    ishares_csv_list = ["./slv_history.csv"]
    df_ishares = None
    for i in range(len(ishares_symbol_list)):
        url = ishares_csv_list[i]
        isym = ishares_symbol_list[i]
        df_temp = pd.read_csv(url)
        df_temp['symbol'] = isym
        if df_ishares is None:
            df_ishares = df_temp.copy()
        else:
            df_ishares = df_ishares.append(df_temp)
    df_ishares.head() 
    def ishares_date(d):
        try:
            dt = datetime.datetime.strptime(str(d),'%b %d, %Y')
        except:
            return None
        return dt
    df_ishares['date'] = df_ishares['As Of'].apply(ishares_date)
    df_ishares = df_ishares.rename(columns = {'NAV per Share':'nav','Shares Outstanding':'shares'})
    df_ishares = df_ishares[['symbol','date','nav','shares']]
    df_ishares = df_ishares[~df_ishares.date.isnull()].sort_values(['symbol','date'])
    df_ishares.index = list(range(len(df_ishares)))
    df_ishares.shares = df_ishares.shares.apply(lambda s:float(str(s).replace(',','')))
    df_ishares.head()


___
### Retrieve data from ProFunds using the url from accounts.profunds
___

In [ ]:
if CREATE_ETF_DATA:
    profunds_symbol_list = ['UCO']
    df_profunds = None
    for psym in profunds_symbol_list:
        url = f"https://accounts.profunds.com/etfdata/ByFund/{psym}-historical_nav.csv"
        df_temp = pd.read_csv(url)
        df_temp['symbol'] = psym
        if df_profunds is None:
            df_profunds = df_temp.copy()
        else:
            df_profunds = df_profunds.append(df_temp)
    df_profunds.head()
    def profunds_date(d):
        try:
            dt = datetime.datetime.strptime(str(d),'%m/%d/%Y')
        except:
            return None
        return dt
    df_profunds.columns.values,df_profunds.head()
    df_profunds['date'] = df_profunds.Date.apply(profunds_date)
    df_profunds = df_profunds.rename(columns = {'NAV':'nav','Shares Outstanding (000)':'shares'})
    df_profunds = df_profunds[['symbol','date','nav','shares']]
    df_profunds = df_profunds[~df_profunds.date.isnull()].sort_values(['symbol','date'])
    df_profunds.index = list(range(len(df_profunds)))
    df_profunds.shares = df_profunds.shares.apply(lambda s:float(str(s).replace(',','')))
    df_profunds.head()
    

___
### Retrieve ETF data for multiple ETFs from us.spdrs.com
___

In [ ]:

if CREATE_ETF_DATA:
    spdr_symbol_list = ['GLD','SPY','XLB','XLE','XLF','XLI','XLK','XLP','XLU']
    df_spdr = None
    for ssym in spdr_symbol_list:
        url = f'https://us.spdrs.com/site-content/xls/{ssym}_HistoricalNav.xls?fund={ssym}&docname=Most+Recent+NAV+%2F+NAV+History&onyx_code1=&onyx_code2='
    #     df_temp = pd.read_excel('https://us.spdrs.com/site-content/xls/SPY_HistoricalNav.xls?fund=SPY&docname=Most+Recent+NAV+%2F+NAV+History&onyx_code1=&onyx_code2=',skiprows=3)
        df_temp = pd.read_excel(url,skiprows=3)
        df_temp['symbol'] = ssym
        if df_spdr is None:
            df_spdr = df_temp.copy()
        else:
            df_spdr = df_spdr.append(df_temp)
    def spdr_date(d):
        try:
            dt = datetime.datetime.strptime(str(d),'%d-%b-%Y')
        except:
            return None
        return dt
    df_spdr['date'] = df_spdr.Date.apply(spdr_date)
    df_spdr = df_spdr.rename(columns = {'Nav':'nav','Shares Outstanding':'shares'})
    df_spdr = df_spdr[['symbol','date','nav','shares']]
    df_spdr = df_spdr[~df_spdr.date.isnull()].sort_values(['symbol','date'])
    df_spdr.index = list(range(len(df_spdr)))
    df_spdr.shares = df_spdr.shares.apply(lambda s:float(str(s).replace(',','')))
    df_spdr.head()

___
### Combine the 3 separate dataframes into one and save it
1. Append df_ishares, df_profunds and df_spdr rows together;
2. Save the combined DataFrame to ./etf_cap_hist.csv;
3. Read that csv back into the DataFrame dff
___

In [ ]:

if CREATE_ETF_DATA:
    df_all = df_ishares.copy()
    df_all = df_all.append(df_profunds,ignore_index=True)
    df_all = df_all.append(df_spdr,ignore_index=True)
    df_all.index = list(range(len(df_all)))
    df_all.to_csv(etf_save_path,index=False)


___
### Create the following columns:
1. pc: shares daily percent change
2. share_diff: daily difference from the shares column
___

In [ ]:
dff  = pd.read_csv(etf_save_path)
dff['date'] = dff.date.apply(str_to_date)
df_fund_flows = dff.copy()
df_fund_flows['shares'] = df_fund_flows.shares.apply(lambda s:float(str(s).replace(',','')))
df_fund_flows = df_fund_flows.sort_values(['symbol','date'])
symbol_list = list(set(df_fund_flows.symbol))
df_final = None
for sym in symbol_list:
    df_this_sym= df_fund_flows[df_fund_flows.symbol==sym]
    df_this_sym['pc'] = df_this_sym.shares.pct_change()
    df_this_sym['share_diff'] = df_this_sym.shares.diff()
    df_this_sym = df_this_sym[df_this_sym.pc.notnull()]
    if df_final is None:
        df_final = df_this_sym.copy()
    else:
        df_final = df_final.append(df_this_sym)

In [ ]:
df_final.tail()

___
### Create the group_by Dataframes:
1. df_pc_gb: shows the min and max per symbol of the pc column;
2. df_share_diff_gb: shows the min and max per symbol of the share_diff column.
___

In [ ]:
dt_beg = datetime.datetime.now() - datetime.timedelta(1000)
df_final2 = df_final[(df_final.date>dt_beg)]
df_pc = df_final2[['symbol','pc']] 
df_pc_gb = df_pc.groupby('symbol',as_index=False).agg({'pc':[min,max]})
df_pc_gb.columns = ['symbol'] + [ t[0]+ '_' + t[1] for t in df_pc_gb.columns.values[1:]]
df_share_diff = df_final2[['symbol','share_diff']] 
df_share_diff_gb = df_share_diff.groupby('symbol',as_index=False).agg({'share_diff':[min,max]})
df_share_diff_gb.columns = ['symbol'] + [ t[0]+ '_' + t[1] for t in df_share_diff_gb.columns.values[1:]]
df_pc_gb,df_share_diff_gb

In [ ]:
df_final2.columns.values

___
### Create a DataFrame that has the pc of each security in a separate column.
This dataframe will make it easy to graph histograms of the pc values
___

In [ ]:
df_pc_all = None
for sym in list(set(df_final2.symbol)):
    df_temp = df_final2[df_final2.symbol==sym][['date','pc']]
    df_temp = df_temp.rename(columns={'pc':sym})
    if df_pc_all is None:
        df_pc_all = df_temp.copy()
    else:
        df_pc_all = df_pc_all.merge(df_temp,how='inner',on='date')
df_pc_all.index = df_pc_all.date
df_pc_all = df_pc_all[list(filter(lambda c:'date' not in c,df_pc_all.columns.values))]

In [ ]:
df_pc_all.columns.values,df_pc_all.as_matrix().shape

___
### Graph histograms of the pc column
___

In [ ]:
df_pc_all.hist(bins=100,figsize=(20,16))

### Show frequency of outlier changes in shares outstanding in GLD

In [ ]:
djan1 = datetime.datetime(2018,1,1)
djan2 = datetime.datetime(2019,2,28)
c1 = df_final2.symbol=='GLD'
c2 = df_final2.date>=djan1
c3 = df_final2.date<=djan2
c4 = df_final2.pc >= .009
allc = (c1) & (c2) & (c3) & (c4)
df_final2[allc]

___
## Process CFTC COT Data
___

### Initial processing
1. Download zip files from www.cft.gov/files;
2. Unip the files using the zipfile package;
3. Read each csv (usually named Annual.TXT), and merge them into the df_cot DataFrame.


In [ ]:
if CREATE_COT_DATA:
    year_list = np.linspace(2000,2019,20)
#     directory_to_extract_to = TEMP_FOLDER
    zip_download_folder = f'TEMP_FOLDER/zip'
#     annual_path = f'{zip_download_folder}/Annual.TXT'
    df_cot_temp = None
    df_cot = None
    for y in year_list:
        yint = int(y)
        url = f"https://www.cftc.gov/files/dea/history/deacot{yint}.zip"
        path_to_zip_file = f'{zip_download_folder}/dea_fut_xls_{y}.zip'
        print(f'retrieving cot zip file from {url}')
        try:
            urllib.request.urlretrieve(url, path_to_zip_file)    
        except:
            import time
            time.sleep(1)
            urllib.request.urlretrieve(url, path_to_zip_file)    
        zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
        zip_ref.extractall(zip_download_folder)
        zip_ref.close()
        df_cot_temp = pd.read_csv(f'{zip_download_folder}/Annual.TXT')
        if df_cot is None:
            df_cot = df_cot_temp.copy()
        else:
            df_cot = df_cot.append(df_cot_temp,ignore_index=True)
            df_cot.index = list(range(len(df_cot)))
        print(f'processed cot csv file from {url}. Length = {len(df_cot_temp)}')    
    df_cot.head()

___
### Make column names easier to process, make main date field a datetime object, and sort the DataFrame
___

In [ ]:
if CREATE_COT_DATA:
    col_rename_dict = {c:c.replace(' ','_').replace('-','_').replace('(','').replace(')','') for c in df_cot.columns.values}
    df_cot2 = df_cot.rename(columns=col_rename_dict)
    df_cot2.As_of_Date_in_Form_YYYY_MM_DD = df_cot2.As_of_Date_in_Form_YYYY_MM_DD.apply(str_to_date)
    df_cot2 = df_cot2.sort_values(['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD'])
    df_cot2.columns.values
    df_cot2.to_csv(cot_save_path,index=False)

___
### Show important columns for a specific  commodity
___

In [ ]:
df_cot2 = pd.read_csv(cot_save_path)
df_cot2.As_of_Date_in_Form_YYYY_MM_DD = df_cot2.As_of_Date_in_Form_YYYY_MM_DD.apply(str_to_date)
# commod = 'CRUDE OIL, LIGHT SWEET'
commod = 'GOLD'
cot_beg_date = datetime.datetime.now() - datetime.timedelta(2000)
df_commod = df_cot2[df_cot2.Market_and_Exchange_Names.str.contains(commod)][df_cot2.As_of_Date_in_Form_YYYY_MM_DD>=cot_beg_date]
basic_cols = ['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD','Open_Interest_All']
long_cols = ['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD',
            'Noncommercial_Positions_Long_All','Commercial_Positions_Long_All',
            'Nonreportable_Positions_Long_All','Traders_Commercial_Long_All',
             'Traders_Noncommercial_Long_All','Traders_Total_Reportable_Long_All']
short_cols = ['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD',
            'Noncommercial_Positions_Short_All','Commercial_Positions_Short_All',
            'Nonreportable_Positions_Short_All','Total_Reportable_Positions_Short_All',
            'Traders_Commercial_Short_All','Traders_Noncommercial_Short_All',
            'Traders_Total_Reportable_Short_All']
df_commod_basic = df_commod[basic_cols]
df_commod_long = df_commod[long_cols]
df_commod_short = df_commod[short_cols]


### Show basic open interest info

In [ ]:
df_commod_basic.head()

### Show important "long" position info

In [ ]:
df_commod_long.head()

### Show important "short" position info

In [ ]:
df_commod_short.head()

### Show important "net" position info

In [ ]:
def non_comm_net(r):
    return float(r.Noncommercial_Positions_Long_All) - float(r.Noncommercial_Positions_Short_All)
def comm_net(r):
    return float(r.Commercial_Positions_Long_All) - float(r.Commercial_Positions_Short_All)
def non_report_net(r):
    return float(r.Nonreportable_Positions_Long_All) - float(r.Nonreportable_Positions_Short_All)
def traders_comm_net(r):
    return float(r.Traders_Commercial_Long_All) - float(r.Traders_Commercial_Short_All)
def traders_noncomm_net(r):
    return float(r.Traders_Noncommercial_Long_All) - float(r.Traders_Noncommercial_Short_All)

df_commod_net = df_commod_long.merge(df_commod_short,how='inner',on=['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD'])
df_commod_net['Noncommercial_Positions_Net_All'] = df_commod_net.apply(non_comm_net,axis=1)
df_commod_net['Commercial_Positions_Net_All'] = df_commod_net.apply(comm_net,axis=1)
df_commod_net['Nonreportable_Positions_Net_All'] = df_commod_net.apply(non_report_net,axis=1)
df_commod_net['Traders_Commercial_Net_All'] = df_commod_net.apply(traders_comm_net,axis=1)
df_commod_net['Traders_Noncommercial_Net_All'] = df_commod_net.apply(traders_noncomm_net,axis=1)
net_cols = ['Market_and_Exchange_Names','As_of_Date_in_Form_YYYY_MM_DD','Noncommercial_Positions_Net_All','Commercial_Positions_Net_All','Nonreportable_Positions_Net_All','Traders_Commercial_Net_All','Traders_Noncommercial_Net_All']
df_commod_net = df_commod_net[net_cols]
df_commod_net.tail()


___
## Merge ETF shares data with COT data
___

### Chose contract and limit dates

In [ ]:
etf_symbol = "GLD"
etf_cond_1 = df_final.symbol==etf_symbol
etf_cond_2 = df_final.date>=cot_beg_date
etf_cond_all = (etf_cond_1 & etf_cond_2)
df_final3 = df_final[etf_cond_all]
df_final3.index = list(range(len(df_final3)))
df_final3.head()

### Merge with gold cot data

In [ ]:
import pandasql as psql

In [ ]:
df_commod_net2 = df_commod_net.copy()
df_commod_net2['next_date'] = df_commod_net2.shift(-1).As_of_Date_in_Form_YYYY_MM_DD
df_commod_net2[df_commod_net2.next_date.isnull()] = datetime.datetime.now() + datetime.timedelta(1)

q = """select * from df_final3
join df_commod_net2 on df_final3.date >= df_commod_net2.As_of_Date_in_Form_YYYY_MM_DD 
     and df_final3.date < df_commod_net2.next_date 
"""
pysqldf = lambda q: psql.sqldf(q, globals())
df_net = pysqldf(q)
df_net2 = df_net[['symbol','date','As_of_Date_in_Form_YYYY_MM_DD','nav',
                  'shares','pc','share_diff','Noncommercial_Positions_Net_All',
                  'Commercial_Positions_Net_All','Nonreportable_Positions_Net_All']]
df_net2 = df_net2.rename(columns={
    'As_of_Date_in_Form_YYYY_MM_DD':'cot_date',
    'Noncommercial_Positions_Net_All':'spec_large','Commercial_Positions_Net_All':'trade',
    'Nonreportable_Positions_Net_All':'spec_small'})
df_net2['fut_diff'] = df_net2.share_diff/ 1000 
df_net3 = df_net2[['symbol','cot_date','fut_diff','spec_large','spec_small']]
df_net3.fut_diff = df_net3.fut_diff.astype(float) 
df_net3.spec_large = df_net3.spec_large.astype(float) 
df_net3.spec_small = df_net3.spec_small.astype(float) 
df_net3['spec'] = df_net3.spec_large + df_net3.spec_small
f = {'fut_diff':sum,'spec':np.mean,'spec_large':np.mean,'spec_small':np.mean}
df_net3_agg = df_net3.groupby(['symbol','cot_date'],as_index=False).agg(f)
df_net3_agg

In [ ]:
dates_per_plot = 40
plots = int(len(df_net3_agg)/dates_per_plot) + 1 if len(df_net3_agg) % dates_per_plot > 0 else 0

# fig, axes = plt.subplots(nrows=plots, ncols=1)
df_net4_agg = df_net3_agg.copy()
f = plt.figure()
def int_date(d):
    return int(str(d)[0:4]+str(d)[5:7]+str(d)[8:10])
df_net4_agg.index = df_net4_agg.cot_date.apply(int_date)
image_names = []
for p in range(plots):
    low_row = p * dates_per_plot
    high_row = low_row + dates_per_plot
    ax = df_net4_agg[['fut_diff','spec',]].iloc[low_row:high_row].plot.bar(figsize=(14,6))
    fig = ax.get_figure()
    image_name = f"./temp_folder/cot/cot_net_{p+1}.png"
    fig.savefig(image_name)
    image_names.append(image_name)


### Merge images into 1 vertically

In [ ]:
imgs    = [ Image.open(i) for i in image_names ]
# pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

# for a vertical stacking it is simple: use vstack
# imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
imgs_comb = np.vstack( (np.asarray(i) for i in imgs ) )
imgs_comb = Image.fromarray( imgs_comb)
save_file = f"./temp_folder/cot/cot_net.png"
imgs_comb.save( save_file)

## END